# Escolhendo Modelos

Agora que já vimos o básico sobre a preparação dos dados, vamos ver como escolher um modelo para a classficação ou predição do nosso conjunto de dados.

In [1]:
using Queryverse
using MLJ

In [4]:
dados = Queryverse.load("iris.csv") |> DataFrame
first(dados, 3)

,sepallength,sepalwidth,petallength,petalwidth,variety
,Float64,Float64,Float64,Float64,String
1,5.1,3.5,1.4,0.2,Setosa
2,4.9,3.0,1.4,0.2,Setosa
3,4.7,3.2,1.3,0.2,Setosa


Verificaremos os tipos científicos que serão utilizados pelo nosso conjunto de dados

In [5]:
schema(dados)

┌─────────────┬─────────┬────────────┐
│ _.names     │ _.types │ _.scitypes │
├─────────────┼─────────┼────────────┤
│ sepallength │ Float64 │ Continuous │
│ sepalwidth  │ Float64 │ Continuous │
│ petallength │ Float64 │ Continuous │
│ petalwidth  │ Float64 │ Continuous │
│ variety     │ String  │ Textual    │
└─────────────┴─────────┴────────────┘
_.nrows = 150


Assim como no notebook anterior mudaremos o tipo da nossa classe __variety__ para um tipo mais adequado para classificação.

In [7]:
coerce!(dados, autotype(dados))
schema(dados)

┌─────────────┬───────────────────────────┬───────────────┐
│ _.names     │ _.types                   │ _.scitypes    │
├─────────────┼───────────────────────────┼───────────────┤
│ sepallength │ Float64                   │ Continuous    │
│ sepalwidth  │ Float64                   │ Continuous    │
│ petallength │ Float64                   │ Continuous    │
│ petalwidth  │ Float64                   │ Continuous    │
│ variety     │ CategoricalString{UInt32} │ Multiclass{3} │
└─────────────┴───────────────────────────┴───────────────┘
_.nrows = 150


Agora que nosso dado está com os tipos científicos adequados vamos separar em dados de entrada e saída.

In [9]:
y, x = unpack(dados, ==(:variety), colname -> true);

No pacote MLJ, um modelo (_**model**_) é uma struct que guarda os hyperparâmetros dos algoritmos de aprendizagem utilizados pelo pacote. Com o conjunto de dados (x, y) e o uso das funções __model__ e __matching__ do pacote MLJ, podemos verificar quais modelos são adequados para os tipos científicos do nosso conjunto de dados.

**Obs: para usar um modelo é necessário ter instalado o pacote do qual ele depende.**

In [15]:
modelos = models(matching(x, y));
length(modelos)

41

Temos um total de 41 modelos disponíveis para uso com o nosso conjunto de dados. Vamos verificar melhor as propriedades do tipo modelo, para isto vamos pegar um dos modelos retornado na nossa pesquisa.

In [18]:
modelos[5]

Bayesian Linear Discriminant Analysis.
→ based on [ScikitLearn](https://github.com/cstjean/ScikitLearn.jl).
→ do `@load BayesianLDA pkg="ScikitLearn"` to use the model.
→ do `?BayesianLDA` for documentation.
(name = "BayesianLDA",
 package_name = "ScikitLearn",
 is_supervised = true,
 docstring = "Bayesian Linear Discriminant Analysis.\n→ based on [ScikitLearn](https://github.com/cstjean/ScikitLearn.jl).\n→ do `@load BayesianLDA pkg=\"ScikitLearn\"` to use the model.\n→ do `?BayesianLDA` for documentation.",
 hyperparameter_ranges = (nothing, nothing, nothing, nothing, nothing, nothing),
 hyperparameter_types = ("String", "Union{Nothing, Float64, String}", "Union{Nothing, AbstractArray{T,1} where T}", "Union{Nothing, Int64}", "Bool", "Float64"),
 hyperparameters = (:solver, :shrinkage, :priors, :n_components, :store_covariance, :tol),
 implemented_methods = [:predict, :clean!, :fit, :fitted_params],
 is_pure_julia = false,
 is_wrapper = false,
 load_path = "MLJModels.ScikitLearn_.Bayes

Vemos muitas informações mas aqui queremos destacar as que consideramos mais importantes inicialmente: 
  - **name** -> nome do modelo dentro do pacote MLJ;
  - **package_name** -> nome do pacote que deve ser instalado para que o modelo possa ser utilizado;
  - **is_supervised** -> informa se o modelo é do tipo supervisionado;
  - **prediction_type** -> informa o tipo de predição, alguns dos tipo disponíveis são:
    - determinístico (deterministic);
    - probabilístico (probabilistic);
    - desconhecido (unknows).
    
Com base no seu conjunto de dados voê deve determinar qual abordagem é melhor para você. Vamos supor que você quer que os seus dados sejam usados em uma abordagem probabilística, para isso vamos fazer o uso novamente das funções __models__ e __matching__ e depois separar somente as abordagens probabilísticas.     

In [30]:
for m in models(matching(x, y))
    if m.prediction_type == :probabilistic
        println(rpad(m.name, 28), "($(m.package_name))")
    end
end

AdaBoostClassifier          (ScikitLearn)
AdaBoostStumpClassifier     (DecisionTree)
BaggingClassifier           (ScikitLearn)
BayesianLDA                 (MultivariateStats)
BayesianLDA                 (ScikitLearn)
BayesianQDA                 (ScikitLearn)
BayesianSubspaceLDA         (MultivariateStats)
ConstantClassifier          (MLJModels)
DecisionTreeClassifier      (DecisionTree)
DummyClassifier             (ScikitLearn)
EvoTreeClassifier           (EvoTrees)
ExtraTreesClassifier        (ScikitLearn)
GaussianNBClassifier        (NaiveBayes)
GaussianNBClassifier        (ScikitLearn)
GaussianProcessClassifier   (ScikitLearn)
GradientBoostingClassifier  (ScikitLearn)
KNNClassifier               (NearestNeighbors)
KNeighborsClassifier        (ScikitLearn)
LDA                         (MultivariateStats)
LinearBinaryClassifier      (GLM)
LogisticCVClassifier        (ScikitLearn)
LogisticClassifier          (MLJLinearModels)
LogisticClassifier          (ScikitLearn)
MultinomialClassifi

Agora temos o nome de uma série de modelos do tipo probabilístico que podem ser utilizados para o nosso conjunto de dados, bem como, do nome do pacote que precisa ser instalado para usar o modelo.

O que precisamos fazer agora é carregar o modelo desejado com a macro _**@load**_.

In [33]:
knc = @load KNNClassifier

┌ Info: A model type "KNNClassifier" is already loaded. 
│ No new code loaded. 
└ @ MLJModels /home/phelipe/.julia/packages/MLJModels/zduEi/src/loading.jl:43


KNNClassifier(
    K = 5,
    algorithm = :kdtree,
    metric = Distances.Euclidean(0.0),
    leafsize = 10,
    reorder = true,
    weights = :uniform) @ 1…77

Em alguns casos podemos ter diferentes pacotes oferecendo o mesmo modelo, como é o caso do modelo __RandomForestClassifier__ que é oferecido tanto pelo pacote __ScikitLearn__ quanto pelo pacote __DecisionTree__, neste caso podemos adicionar o nome do pacote junto do comando _**@load**_.

In [38]:
rfc = @load  RandomForestClassifier pkg="DecisionTree"

RandomForestClassifier(
    max_depth = -1,
    min_samples_leaf = 1,
    min_samples_split = 2,
    min_purity_increase = 0.0,
    n_subfeatures = 0,
    n_trees = 10,
    sampling_fraction = 0.7,
    pdf_smoothing = 0.0) @ 7…68

Neste notebook vimos um pouco sobre como utilizar os modelos, no próximo notebook veremos como utilizar os modelos.